#Spleeter Colab

Esta es una extensión para [Deezer Spleeter](https://github.com/deezer/spleeter) que se ejecuta en Google Colab, utilizando Google Drive como almacenamiento de datos.

El valor de `input` puede ser
1.   Ruta a un archivo de audio individual ubicado en su Google Drive.
2.   Ruta a un directorio en su Google Drive que contenga varios archivos de audio. Cada archivo de audio encontrado en el directorio será procesado individualmente.
3.   URL individual de YouTube
4.   Múltiples URLs de YouTube separadas por espacios. Cada vídeo de YouTube será procesado individualmente.

La entrada será separada en 4 stems: voz, batería, bajo y otros. Los stems se guardarán como archivos WAV en su Google Drive. Introduzca `output_dir` relativo a su raíz de Google Drive.

In [2]:
#@title Setup
#@markdown This cell needs to be run only once. It will mount your Google Drive and setup prerequisites.

import os
from google.colab import drive, output
drive.mount('/content/drive')
drive_root = '/content/drive/MyDrive/'

# inhagcutils
!pip -q install import-ipynb
!curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *
!gsutil -q -m cp -R gs://neural-research/olaviinha/spleeter-configs/custom-4stems-22kHz-z.json /content/cfg.json

#@title Install
!apt -q install ffmpeg
!pip -q install spleeter

output.clear()
print('Using config gs://neural-research/olaviinha/spleeter-configs/custom-4stems-22kHz-z.json')
op(c.ok, 'Setup finished.')


Using config gs://neural-research/olaviinha/spleeter-configs/custom-4stems-22kHz-z.json
Setup finished.


In [3]:
#@title Separate

input = "/content/[Hi-Res] ME!ME!ME! CHRONIC feat.daoko (TeddyLoid Mega Remix).flac" #@param {type:"string"}
output_dir = "/sa" #@param {type:"string"}

#----------------------------------------------------------------------------#

import os
from os.path import isfile, join
from glob import glob

input_type = check_input_type(input)
tmp_dir = '/content/tmp/'
seps_dir = fix_path(drive_root+output_dir)
#seps_dir = '/content/tmp/seps'

if not os.path.isdir(tmp_dir):
  create_dirs([tmp_dir])
# if not os.path.isdir(output_dir):
#   create_dirs([seps_dir])

if output_dir == '' or output_dir == 'same':
  output_dir = fix_path(drive_root+path_dir(input))
else:
  output_dir = fix_path(drive_root+output_dir)
  if not os.path.isdir(output_dir):
    create_dirs([output_dir])

if input_type == 'unknown':
  input = drive_root+input
  input_type = check_input_type(input)
  if input_type=='dir':
    input = fix_path(input)

if input_type == 'youtube':
  !pip -q install youtube-dl
  !youtube-dl --restrict-filenames -x --no-continue --audio-format wav -o "{tmp_dir}%(title)s.%(ext)s" {input}

from spleeter.separator import Separator
separator = Separator('/content/cfg.json')

if input_type == "dir" or input_type == "youtube":
  if input_type == "dir":
    target = input
  if input_type == "youtube":
    target = dir_tmp
  file_list = list_audio(target)
  for audiofile in file_list:
    op(c.title, '\nSeparate '+audiofile+' to '+seps_dir.replace(drive_root, '')+basename(input))
    # Spleeter Python API is outdated for Colab
    # separator.separate_to_file(audiofile, seps_dir)
    !spleeter separate -p /content/cfg.json -o "{output_dir}" "{audiofile}"
    op(c.ok, 'FIN.', 'Files saved under directory '+output_dir.replace(drive_root, '')+basename(audiofile)+'/')
elif input_type == "file":
  # Spleeter Python API is outdated for Colab
  # separator.separate_to_file(input, seps_dir)
  !spleeter separate -p /content/cfg.json -o "{seps_dir}" "{input}"
  op(c.ok, 'FIN.', 'Files saved under directory '+output_dir.replace(drive_root, '')+basename(input)+'/')
else:
  print("We're not sure what you want to separate. Please check the input field.")

tcmalloc: large alloc 1080041472 bytes == 0x729dc000 @  0x7fbb127211e7 0x7fbb0eedb46e 0x7fbb0ef2bc7b 0x7fbb0ef2bd18 0x7fbb0efd3010 0x7fbb0efd373c 0x7fbb0efd385d 0x4bc4ab 0x7fbb0ef18ef7 0x59371f 0x515244 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x511e2c 0x593dd7 0x511e2c 0x549576 0x593fce 0x511e2c 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x4bca8a 0x5134a6 0x549e0e
INFO:spleeter:Downloading model archive https://github.com/deezer/spleeter/releases/download/v1.4.0/4stems.tar.gz
INFO:spleeter:Validating archive checksum
INFO:spleeter:Extracting downloaded 4stems archive
INFO:spleeter:4stems model file(s) extracted
tcmalloc: large alloc 1730199552 bytes == 0xd3fde000 @  0x7fbb127211e7 0x7fbadbdd3f42 0x7fbade7d8c5a 0x7fbadeb795c4 0x7fbadebc9b1f 0x7fbadebcb769 0x7fbadebcc757 0x7fbadebcde6e 0x7fbadebd2942 0x7fbadebd2e3f 0x7fbad4a72e54 0x7fbad4a671be 0x7fbad9f1d021 0x7fbad9f19d33 0x7fbad51af8b5 0x7fbb120e56db 0x7fbb1241e61f
tcmalloc: large alloc 1730199552 bytes == 0x13b1ea000 